# Esh: Non-Monotonic Activation Function


## Install Packages

In [ ]:
!pip install -q wandb
!pip install -q kaggle
!pip install -q torchvision
!pip install -q pycocotools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Change Current Working Directory
!mkdir -p data/coco
!unzip -q /content/gdrive/MyDrive/ImgSeg/train2017.zip -d data/coco/train
!unzip -q /content/gdrive/MyDrive/ImgSeg/val2017.zip -d data/coco/val
!unzip -q /content/gdrive/MyDrive/ImgSeg/annotations_trainval2017.zip -d data/coco/annotations

In [ ]:
!git clone https://github.com/Suresh-Rajendran/unet_poc.git unet_coco_v1

Cloning into 'unet_coco_v1'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 104 (delta 52), reused 78 (delta 26), pack-reused 0
Receiving objects: 100% (104/104), 167.61 KiB | 10.47 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [32]:
import os
os.chdir('/content/unet_coco_v1')

# Hyperparameter Sweeps

In [34]:
# Login to WandB
!wandb login a96eb1d17152292d104d045304f8da02ab8fe0e1

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#import wandb
#wandb.init(project="Esh_UNET_v2", entity="pes_mtech_jun21_g3")

In [38]:
# Imports the parameter configuration for the sweep run. To add more hyperparameters in the suite, simply edit the sweep.yaml file.
!wandb sweep esh_unet_train_sweep_coco.yaml    # Outputs the Sweep ID

wandb: Creating sweep from: esh_unet_train_sweep_coco.yaml
wandb: Created sweep with ID: b50odsk0
wandb: View sweep at: https://wandb.ai/pes_mtech_jun21_g3/unet_coco_v1/sweeps/b50odsk0
wandb: Run sweep agent with: wandb agent pes_mtech_jun21_g3/unet_coco_v1/b50odsk0


In [36]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [39]:
!wandb agent pes_mtech_jun21_g3/unet_coco_v1/b50odsk0

wandb: Starting wandb agent 🕵️
2023-04-17 08:14:06,818 - wandb.wandb_agent - INFO - Running runs: []
2023-04-17 08:14:07,194 - wandb.wandb_agent - INFO - Agent received command: run
2023-04-17 08:14:07,195 - wandb.wandb_agent - INFO - Agent starting run with config:
	act: relu
	batch_size: 8
	epochs: 2
	optimizer: Adam
	version: unet
2023-04-17 08:14:07,196 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python train_unet_coco.py --act=relu --batch_size=8 --epochs=2 --optimizer=Adam --version=unet
wandb: Currently logged in as: rsuresh002 (pes_mtech_jun21_g3). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: Tracking run with wandb version 0.14.2
wandb: Run data is saved locally in /content/unet_coco_v1/wandb/run-20230417_081410-q4yghy0f
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run zany-sweep-1
wandb: ⭐️ View project at https://wandb.ai/pes_mtech_jun21_g3/unet_coco_v1
wandb: 🧹

KeyboardInterrupt: ignored

In [44]:
import numpy as np
import cv2
from pycocotools.coco import COCO

# Path to the COCO dataset annotations file
ann_file = '/content/data/coco/annotations/annotations/instances_train2017.json'

# Path to the COCO dataset images folder
img_folder = '/content/data/coco/train/train2017/'

# Load the COCO annotations
coco = COCO(ann_file)

# Get the category IDs and names
cat_ids = coco.getCatIds()
cat_names = [coco.loadCats(id)[0]['name'] for id in cat_ids]

# Define the RGB colors for each category
colors = np.random.randint(0, 255, (len(cat_ids), 3), dtype=np.uint8)

# Iterate over the images
img_size = 0
for img_id in coco.imgs:
    # Load the image
    if img_size > 5000:
      break
    img_path = img_folder + '/' + coco.imgs[img_id]['file_name']
    img = cv2.imread(img_path)

    # Create a binary mask for each object category
    binary_masks = []
    for ann in coco.imgToAnns[img_id]:
        cat_id = ann['category_id']
        mask = coco.annToMask(ann)
        binary_mask = np.zeros_like(mask, dtype=np.uint8)
        binary_mask[mask == 1] = 1
        binary_masks.append((binary_mask, cat_id))

    # Combine the binary masks into a single mask
    mask = np.zeros_like(binary_mask, dtype=np.uint8)
    for binary_mask, cat_id in binary_masks:
        mask[binary_mask == 1] = cat_id

    # Resize the mask to match the size of the image
    mask = cv2.resize(mask, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Convert the mask to an RGB mask
    rgb_mask = np.zeros_like(img)
    for i, color in enumerate(colors):
        rgb_mask[mask == cat_ids[i]] = color

    # Save the RGB mask
    os.makedirs('/content/data/coco/mask/', exist_ok=True)
    mask_path = '/content/data/coco/mask/' + coco.imgs[img_id]['file_name']
    cv2.imwrite(mask_path, rgb_mask)
    img_size +=1


loading annotations into memory...
Done (t=16.28s)
creating index...
index created!


In [31]:
import os
import shutil
from PIL import Image, ImageChops
from pycocotools.coco import COCO
import numpy as np
import random


def create_colormap(max_cat_id):
    random.seed(42)  # Set the random seed for reproducibility
    colormap = np.zeros((max_cat_id + 1, 3), dtype=int)
    for i in range(max_cat_id + 1):
        colormap[i] = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
    
    colormap[0] = [0,0,0]
    return colormap


def apply_colormap(mask, colormap):
    h, w = mask.shape
    #print(mask)
    colored_mask = np.zeros((h, w, 3), dtype=np.uint8)
    for i in range(h):
      for j in range(w):
        c_idx = mask[i, j] if mask[i, j] < colormap.shape[0] else 0
        #if mask[i, j] != c_idx:
          #print( mask[i, j], c_idx)
        colored_mask[i, j] = colormap[c_idx]
    return colored_mask


def prepare_mini_coco(data_dir, imgs_folder, masks_folder, num_samples, colormap, cat_id_mapping):
    os.makedirs(imgs_folder, exist_ok=True)
    os.makedirs(masks_folder, exist_ok=True)

    coco = COCO(os.path.join(data_dir, "annotations/annotations/instances_train2017.json"))
    img_ids = coco.getImgIds()[:num_samples]

    for img_id in img_ids:
        img_data = coco.loadImgs(img_id)[0]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)
        mask = Image.new("L", (img_data["width"], img_data["height"]))

        for ann in anns:
            mask_ann = Image.fromarray((coco.annToMask(ann) * ann["category_id"]).astype("uint8"))
            mask = ImageChops.add(mask, mask_ann)

        colored_mask = apply_colormap(np.array(mask), colormap)

        img_file = os.path.join(data_dir, "train/train2017", img_data["file_name"])
        mask_file = os.path.join(masks_folder, f"{img_data['file_name'].split('.')[0]}_mask.png")

        shutil.copy(img_file, os.path.join(imgs_folder, img_data["file_name"]))
        Image.fromarray(colored_mask).save(mask_file)

data_dir = "data/coco"
imgs_folder = "/content/unet_coco_v1/data/imgs"
masks_folder = "/content/unet_coco_v1/data/masks"
num_samples = 1000

coco = COCO(os.path.join(data_dir, "annotations/annotations/instances_train2017.json"))
cat_ids = coco.getCatIds()
cat_id_mapping = {cat_id: i for i, cat_id in enumerate(cat_ids)}

max_cat_id = max(cat_ids)
print(cat_ids)
colormap = create_colormap(max_cat_id)
print(colormap[0])

prepare_mini_coco(data_dir, imgs_folder, masks_folder, num_samples, colormap, cat_id_mapping)


loading annotations into memory...
Done (t=23.50s)
creating index...
index created!
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90]
[0 0 0]
loading annotations into memory...
Done (t=16.16s)
creating index...
index created!
